# CIS545: Final Project - CORD-19 Dataset
## Justin Choi
## TA: Hoyt Gong

Hi there! For this project, I opted to use the COVID-19 research challenge dataset, which they named "CORD-19" (I promise, the title wasn't a typo haha), and below you'll see the respective EDA and modeling I've done for the dataset! Hope you enjoy ~


In [2]:
import pandas as pd 
import numpy as np 
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import glob, os
import json
import nltk


In [6]:
metadata_df = pd.read_csv('./CORD-19-research-challenge/metadata.csv')

# Import all the json files
cord_19_folder = './CORD-19-research-challenge/'
list_of_files = []; # only going to take those from pdf_json! not pmc_json
for root, dirs, files in os.walk(cord_19_folder):
    for name in files:
        # if (not(name.endswith('.xml.json')) and name.endswith('.json')):
        if (name.endswith('.json')):
            full_path = os.path.join(root, name)
            list_of_files.append(full_path)
sorted(list_of_files)
len(list_of_files)

# ALTERNATE

all_json = glob.glob(f'{cord_19_folder}/**/*.json', recursive=True)
len(all_json)


0